# Set-up

In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes
!pip install -qqq torch
!pip install -qqq -U git+https://github.com/huggingface/transformers.git
!pip install -qqq -U git+https://github.com/huggingface/peft.git
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops
!pip install -qqq mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 853.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import os
from pprint import pprint
# import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import mlflow

import locale
locale.getpreferredencoding = lambda: "UTF-8"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Prep dataset

In [4]:
import pandas as pd
df = pd.read_json("/content/merged_data_clean.json", orient="records", dtype={"tech_level": str})
df

,text,tech_level
0,Maîtrise avancée de Tableau et Python requise.,"{'Tableau': 'Expert', 'Python': 'Expert'}"
1,Connaissance intermédiaire en SQL et Power BI.,"{'SQL': 'Intermédiaire', 'Power BI': 'Interméd..."
2,Expérience avancée avec SAS et R pour l'analys...,"{'SAS': 'Expert', 'R': 'Expert'}"
3,Bonne maîtrise de Git et GitHub.,"{'Git': 'Intermédiaire', 'Github': 'Intermédia..."
4,Niveau débutant en HTML et CSS.,"{'HTML': 'Débutant', 'CSS': 'Débutant'}"
...,...,...
213,"Développeur créatif, explorant les facettes de...","{'REXX': 'Débutant', 'SSH': 'Débutant', 'SQL':..."
214,"Architecte système intermédiaire, jonglant ave...","{'SNMP': 'Intermédiaire', 'ITAC': 'Intermédiai..."
215,"Ingénieur réseau débutant, découvrant les conf...","{'VLAN': 'Débutant', 'ACL': 'Débutant'}"
216,"Analyste IT intermédiaire, maîtrisant les méan...","{'Jira': 'Intermédiaire', 'Bitbucket': 'Interm..."


In [5]:
data = Dataset.from_pandas(df)

In [6]:
data

Dataset({
    features: ['text', 'tech_level'],
    num_rows: 218
})

In [7]:
# Split the dataset train-test
dataset = data.train_test_split(test_size=0.2, seed=87)
# Split train again to get valid
training_dataset = dataset['train'].train_test_split(test_size=0.2, seed=87)

# Now you have a dictionary with 'train' and 'test' keys
train_dataset = training_dataset['train']
valid_dataset = training_dataset['test']
test_dataset = dataset['test']

In [8]:
valid_dataset[10]

{'text': 'Administrateur système compétent en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes.',
 'tech_level': "{'Linux': 'Expert', 'Docker': 'Expert', 'Kubernetes': 'Débutant'}"}

# Base model

## Load base model

In [9]:
# MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"
# MODEL_NAME = "baffo32/decapoda-research-llama-7B-hf"
# MODEL_NAME = "euclaise/falcon_1b_stage1"
# MODEL_NAME = "tiiuae/falcon-rw-1b"
# MODEL_NAME = "mistralai/Mistral-7B-v0.1"
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Setting pad_token in tokenizer and model, as Mistral model has no defined pad token
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = tokenizer.unk_token  # Better to use <UNK> token to avoid confusion with EOS and over generation
model.config.pad_token_id = tokenizer.pad_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Test base model

In [10]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

In [11]:
generation_config = model.generation_config
generation_config.max_new_tokens = 100
# generation_config.temperature = 0.1
# generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
# generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.pad_token_id = tokenizer.pad_token_id  # Setting pad token for model as the tokenizer pad token
generation_config.eos_token_id = tokenizer.eos_token_id
# generation_config.do_sample=True
generation_config.do_sample=False   # going greedy for reproducibility

In [12]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 100,
  "pad_token_id": 0
}

In [13]:
def make_predictions(prompt, model, generation_config):
  encoding = tokenizer(prompt, return_tensors="pt").to(device)

  with torch.inference_mode():
    outputs = model.generate(
        # input_ids = encoding.input_ids,
        # attention_mask = encoding.attention_mask,
        **encoding,
        generation_config = generation_config
    )

  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [14]:
instruct_prompt = f"""
[INST] {valid_dataset[10]['text']} [/INST]
""".strip()
print(instruct_prompt)

[INST] Administrateur système compétent en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes. [/INST]


In [15]:
%%time
device = "cuda:0"

make_predictions(instruct_prompt, model, generation_config)

[INST] Administrateur système compétent en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes. [/INST] As a competent Linux system administrator with advanced expertise in Docker and beginner-level skills in Kubernetes, I can perform the following tasks:

1. Manage and maintain Linux servers: I can install, configure, and update Linux servers, as well as perform routine maintenance tasks such as patching, backups, and monitoring.

2. Manage Docker containers: I can create, deploy, and manage Docker containers on Linux servers. I can also manage
CPU times: user 12.3 s, sys: 666 ms, total: 12.9 s
Wall time: 16.8 s


In [16]:
another_prompt = f"""
[INST] Développeur web spécialisé en front end et dans le web marketing, expert en coding HTML et Javascript, j'ai travaillé pendant plus deux ans au sein d'une entreprise de vente de vêtements en ligne et souhaite aujourd hui approfondir mon expérience au sein des équipes tech de Lullaby. [/INST]
""".strip()
print(another_prompt)


[INST] Développeur web spécialisé en front end et dans le web marketing, expert en coding HTML et Javascript, j'ai travaillé pendant plus deux ans au sein d'une entreprise de vente de vêtements en ligne et souhaite aujourd hui approfondir mon expérience au sein des équipes tech de Lullaby. [/INST]


In [17]:
%%time
device = "cuda:0"

make_predictions(another_prompt, model, generation_config)

[INST] Développeur web spécialisé en front end et dans le web marketing, expert en coding HTML et Javascript, j'ai travaillé pendant plus deux ans au sein d'une entreprise de vente de vêtements en ligne et souhaite aujourd hui approfondir mon expérience au sein des équipes tech de Lullaby. [/INST] I. Intro

You are a front-end web developer with expertise in HTML and JavaScript, having worked for over two years in an online clothing sales company. Now, you aim to deepen your experience by joining the tech teams at Lullaby.

II. Skills

- Front-end web development
- HTML coding
- JavaScript
- Web marketing

III. Work Experience

- Front-end developer at [Online Clothing Sales Company],
CPU times: user 9.09 s, sys: 205 ms, total: 9.3 s
Wall time: 9.38 s


In [18]:
empty_prompt = f"""
[INST] Mes vacances étaient si agréables cette année. J'en ai profité pour acquérir de nouvelles compétences, je commence donc à cuisiner. [/INST]
""".strip()
print(empty_prompt)

[INST] Mes vacances étaient si agréables cette année. J'en ai profité pour acquérir de nouvelles compétences, je commence donc à cuisiner. [/INST]


In [19]:
%%time
device = "cuda:0"

make_predictions(empty_prompt, model, generation_config)

[INST] Mes vacances étaient si agréables cette année. J'en ai profité pour acquérir de nouvelles compétences, je commence donc à cuisiner. [/INST] Your vacations were so pleasant this year. I took advantage of it and started learning to cook.
CPU times: user 2.15 s, sys: 125 ms, total: 2.27 s
Wall time: 2.27 s


# Fine tuned model

## Load adapter

In [20]:
# Login for adapter model
notebook_login()

In [21]:
# Load adapter model and build it from the base model
config = PeftConfig.from_pretrained("vachonni/mistral7b-tech_level-with_adapter_head")
model = PeftModel.from_pretrained(model, "vachonni/mistral7b-tech_level-with_adapter_head")

adapter_config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

In [22]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 100,
  "pad_token_id": 0
}

## Test fine tuned model

In [23]:
%%time
device = "cuda:0"

make_predictions(instruct_prompt, model, generation_config)

[INST] Administrateur système compétent en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes. [/INST]
{'Linux': 'Expert', 'Docker': 'Expert', 'Kubernetes': 'Débutant'}
CPU times: user 4.86 s, sys: 95.1 ms, total: 4.96 s
Wall time: 5.9 s


In [24]:
%%time
device = "cuda:0"

make_predictions(another_prompt, model, generation_config)

[INST] Développeur web spécialisé en front end et dans le web marketing, expert en coding HTML et Javascript, j'ai travaillé pendant plus deux ans au sein d'une entreprise de vente de vêtements en ligne et souhaite aujourd hui approfondir mon expérience au sein des équipes tech de Lullaby. [/INST]
{'HTML': 'Expert', 'Javascript': 'Expert'}
CPU times: user 2.81 s, sys: 327 ms, total: 3.14 s
Wall time: 3.14 s


In [25]:
%%time
device = "cuda:0"

make_predictions(empty_prompt, model, generation_config)

[INST] Mes vacances étaient si agréables cette année. J'en ai profité pour acquérir de nouvelles compétences, je commence donc à cuisiner. [/INST]
{'Cuisson': 'Débutant'}
CPU times: user 2.01 s, sys: 220 ms, total: 2.23 s
Wall time: 2.25 s


# TODO :: Create a HuggingFace pipeline  /  Serving the model efficiently

In [ ]:
# Merging base model and adapter
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:233: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# Check merged model works the same
%%time
device = "cuda:0"

make_predictions(instruct_prompt, merged_model, generation_config)

[INST] Administrateur système compétent en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes. [/INST] I. Intéroduction

Dans ce chapitré, nous allons explorer les compétences d'un administrateur système expérimenté en Linux, avec une expertise approfondie en Docker et des compétences débutantes en Kubernetes.

II. Compétences en Linux

Un administrateur système expérimenté en Linux possède une solide connaissance des concepts de base de l'exploitation de ce syst
CPU times: user 10.1 s, sys: 196 ms, total: 10.3 s
Wall time: 10.9 s


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model = merged_model, tokenizer = tokenizer, return_full_text=False)

In [ ]:
generator(chat_prompt)

[{'generated_text': " {'Linux': 'Expert', 'Docker': 'Expert', 'Kubernetes': 'Débutant'}"}]

In [ ]:
generator.push_to_hub(vachonni/mistral7b-tech_level-merged-strange_works_with_chat)

AttributeError: 'TextGenerationPipeline' object has no attribute 'push_to_hub'

In [ ]:
type(merged_model)

transformers.models.mistral.modeling_mistral.MistralForCausalLM